In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set()

In [2]:
user_ratings=pd.read_csv('user_ratings2.csv')
movies=pd.read_csv('movies3.csv')

In [3]:
user_ratings.head()

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(user_ratings)

670

In [5]:
rows=[]
for index,row in user_ratings.iterrows():
    rows.append([index,row['userId']])
rowsnp=np.array(rows)
userid_index_mapping=pd.DataFrame(data=rowsnp,columns=['index','user_id'])

In [6]:
userid_index_mapping['index']=userid_index_mapping['index'].astype('int64',inplace=True)
userid_index_mapping['user_id']=userid_index_mapping['user_id'].astype('int64',inplace=True)

In [7]:
userid_index_mapping

,index,user_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
5,5,6
6,6,7
7,7,8
8,8,9
9,9,10


In [8]:
user_ratings_uid=user_ratings.set_index('userId')

In [9]:
movies_ids=[int(i) for i in user_ratings_uid.columns]
rows=[]
for i in range(len(movies_ids)):
    rows.append([i,movies_ids[i]])
movieid_index_mapping=pd.DataFrame(data=rows,columns=['index','movieId'])

In [10]:
movieid_index_mapping

,index,movieId
0,0,5
1,1,11
2,2,12
3,3,13
4,4,14
5,5,16
6,6,18
7,7,19
8,8,20
9,9,22


# Content Based Recommendation

## (1) Overview based

In [11]:
user_ratings_uid_copy=user_ratings_uid.copy()

In [12]:
movies.head()

,Unnamed: 0,budget,genres,id,overview,popularity,runtime,original_language,title
0,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,8,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,12,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,15,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [13]:
movies_copy=movies.copy()

In [14]:
overviews=movies['overview']

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
vectorizer=tfidf(stop_words='english',max_df=0.995,min_df=0.005)
overview_matrix=vectorizer.fit_transform(overviews)


In [16]:
overview_matrix.shape

(857, 1017)

In [17]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(overview_matrix,overview_matrix)


In [18]:
sig[0]

array([0.7620068 , 0.76159416, 0.76159416, 0.761642  , 0.76160569,
       0.76161912, 0.76159416, 0.76159416, 0.76160211, 0.76159416,
       0.76161293, 0.76159416, 0.76162524, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.7616185 , 0.76160825, 0.76161655,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76160742, 0.76161204, 0.76159416,
       0.76159416, 0.76161862, 0.76163524, 0.76159416, 0.76160612,
       0.76159416, 0.76159416, 0.76162435, 0.76162151, 0.76159416,
       0.76159416, 0.76159416, 0.76160399, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76160671, 0.76162843, 0.76160286, 0.76159416, 0.76160875,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76161685, 0.76159416, 0.76159416, 0.7616156 ,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159

In [19]:
movies['title'].unique()

array(["Pirates of the Caribbean: At World's End", 'Spider-Man 3',
       'Harry Potter and the Half-Blood Prince',
       "Pirates of the Caribbean: Dead Man's Chest",
       'The Chronicles of Narnia: Prince Caspian',
       'Pirates of the Caribbean: On Stranger Tides',
       'The Golden Compass', 'King Kong', 'Titanic', 'Spider-Man 2',
       'Transformers: Revenge of the Fallen', 'Cars 2',
       'Terminator Salvation',
       'Indiana Jones and the Kingdom of the Crystal Skull',
       'The Dark Knight', 'Iron Man', 'Wild Wild West',
       'The Mummy: Tomb of the Dragon Emperor', 'The Polar Express',
       'Terminator 3: Rise of the Machines', "The Sorcerer's Apprentice",
       'Shrek the Third', 'Transformers', 'Alexander',
       'Harry Potter and the Goblet of Fire',
       'Charlie and the Chocolate Factory', 'Ratatouille',
       'Batman Begins', 'X-Men Origins: Wolverine',
       'The Matrix Revolutions', 'Mission: Impossible III', 'Armageddon',
       'Men in Black II'

In [20]:
movies[movies.title=='King Kong']['id'].tolist()

[254]

In [21]:
def give_overview_recommendations(movie_title,sig):
    movie_id=movies[movies.title==movie_title]['id'].tolist()[0]
    index=movieid_index_mapping[movieid_index_mapping.movieId==movie_id]['index'].tolist()[0]
    similarity=sorted(list(enumerate(sig[index])),key=lambda x:x[1],reverse=True)
    recommended_movie_indices=[i[0] for i in similarity]
    recommended_movies=[]
    for i in recommended_movie_indices[1:11
                                      ]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies
    
    

In [22]:
give_overview_recommendations("Licence to Kill",sig)

['Dracula',
 'Doctor Zhivago',
 'Mirror Mirror',
 'Mad Max 2: The Road Warrior',
 'The Number 23',
 'The Sentinel',
 'The Reaping',
 'Eyes Wide Shut',
 'Hero',
 'Carriers']

## (2) Content Based Recommendations

In [23]:
movies=movies_copy

In [24]:
movies.head()

,Unnamed: 0,budget,genres,id,overview,popularity,runtime,original_language,title
0,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,8,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,12,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,15,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [25]:
movies=movies.iloc[:,1:]

In [26]:
movies.head()

,budget,genres,id,overview,popularity,runtime,original_language,title
0,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End
1,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3
2,250000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince
3,200000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest
4,225000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian


In [27]:
import json
movies['genres']=movies['genres'].apply(json.loads)

In [28]:
movies['genres']

0      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
1      [{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...
2      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
3      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
4      [{'id': 12, 'name': 'Adventure'}, {'id': 10751...
5      [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
6      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
7      [{'id': 12, 'name': 'Adventure'}, {'id': 18, '...
8      [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
9      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
10     [{'id': 878, 'name': 'Science Fiction'}, {'id'...
11     [{'id': 16, 'name': 'Animation'}, {'id': 10751...
12     [{'id': 28, 'name': 'Action'}, {'id': 878, 'na...
13     [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
14     [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...
15     [{'id': 28, 'name': 'Action'}, {'id': 878, 'na...
16     [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
17     [{'id': 12, 'name': 'Adv

In [29]:
def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])
movies['genres'] = movies['genres'].apply(pipe_flatten_names)

In [30]:
movies['genres']

0                             Adventure|Fantasy|Action
1                             Fantasy|Action|Adventure
2                             Adventure|Fantasy|Family
3                             Adventure|Fantasy|Action
4                             Adventure|Family|Fantasy
5                             Adventure|Action|Fantasy
6                                    Adventure|Fantasy
7                               Adventure|Drama|Action
8                               Drama|Romance|Thriller
9                             Action|Adventure|Fantasy
10                    Science Fiction|Action|Adventure
11                   Animation|Family|Adventure|Comedy
12                     Action|Science Fiction|Thriller
13                                    Adventure|Action
14                         Drama|Action|Crime|Thriller
15                    Action|Science Fiction|Adventure
16     Action|Adventure|Comedy|Science Fiction|Western
17                            Adventure|Action|Fantasy
18        

In [31]:
cleaned = movies.genres.str.split('|', expand=True).stack()
dummies=pd.get_dummies(cleaned, prefix='g').groupby(level=0).sum()
movies_encoded=movies.merge(dummies,left_index=True, right_index=True)






In [32]:
movies_encoded.head()

,budget,genres,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,...,g_Foreign,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Science Fiction,g_Thriller,g_War,g_Western
0,300000000,Adventure|Fantasy|Action,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,...,0,0,0,0,0,0,0,0,0,0
1,258000000,Fantasy|Action|Adventure,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,250000000,Adventure|Fantasy|Family,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,...,0,0,0,0,0,0,0,0,0,0
3,200000000,Adventure|Fantasy|Action,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,...,0,0,0,0,0,0,0,0,0,0
4,225000000,Adventure|Family|Fantasy,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
movies=movies_encoded.drop('genres',axis=1)

In [34]:
movies_copy10=movies.copy()

In [35]:
movies

,budget,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,...,g_Foreign,g_History,g_Horror,g_Music,g_Mystery,g_Romance,g_Science Fiction,g_Thriller,g_War,g_Western
0,300000000,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,258000000,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,250000000,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,200000000,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,225000000,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,380000000,1865,Captain Jack Sparrow crosses paths with a woma...,135.413856,136.0,en,Pirates of the Caribbean: On Stranger Tides,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,180000000,2268,"After overhearing a shocking secret, precociou...",42.990906,113.0,en,The Golden Compass,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,207000000,254,"In 1933 New York, an overly ambitious movie pr...",61.226010,187.0,en,King Kong,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,200000000,597,"84 years later, a 101-year-old woman named Ros...",100.025899,194.0,en,Titanic,0,0,0,...,0,0,0,0,0,1,0,1,0,0
9,200000000,558,Peter Parker is going through a major identity...,35.149586,127.0,en,Spider-Man 2,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
overview_tfidf=overview_matrix

In [37]:
overview_cols=vectorizer.get_feature_names()

In [38]:
from pandas import DataFrame

In [40]:
overview_df=pd.DataFrame(overview_tfidf.toarray(),columns=overview_cols)

In [41]:
movies_with_overview=pd.concat([movies,overview_df],axis=1)

In [42]:
movies_with_overview

,budget,id,overview,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,...,works,world,worse,writer,wrong,year,years,york,young,younger
0,300000000,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0
1,258000000,559,The seemingly invincible Spider-Man goes up ag...,115.699814,139.0,en,Spider-Man 3,1,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0
2,250000000,767,"As Harry begins his sixth year at Hogwarts, he...",98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.207012,0.00000,0.000000,0.000000,0.0
3,200000000,58,Captain Jack Sparrow works his way out of a bl...,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,...,0.388641,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0
4,225000000,2454,One year after their incredible adventures in ...,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.184659,0.00000,0.000000,0.148494,0.0
5,380000000,1865,Captain Jack Sparrow crosses paths with a woma...,135.413856,136.0,en,Pirates of the Caribbean: On Stranger Tides,1,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0
6,180000000,2268,"After overhearing a shocking secret, precociou...",42.990906,113.0,en,The Golden Compass,0,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0
7,207000000,254,"In 1933 New York, an overly ambitious movie pr...",61.226010,187.0,en,King Kong,1,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.216251,0.000000,0.0
8,200000000,597,"84 years later, a 101-year-old woman named Ros...",100.025899,194.0,en,Titanic,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.127327,0.12066,0.000000,0.102390,0.0
9,200000000,558,Peter Parker is going through a major identity...,35.149586,127.0,en,Spider-Man 2,1,1,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0


In [43]:
movies_with_overview=movies_with_overview.drop('overview',axis=1)

In [44]:
movies_with_overview.columns


Index(['budget', 'id', 'popularity', 'runtime', 'original_language', 'title',
       'g_Action', 'g_Adventure', 'g_Animation', 'g_Comedy',
       ...
       'works', 'world', 'worse', 'writer', 'wrong', 'year', 'years', 'york',
       'young', 'younger'],
      dtype='object', length=1042)

In [45]:
from sklearn.preprocessing import MinMaxScaler

In [49]:
user_ratings_nonan=user_ratings.fillna(int(0))

In [50]:
user_ratings_nonan

,userId,5,11,12,13,14,16,18,19,20,...,64499,69640,74510,77866,89492,98369,103731,114635,115210,116977
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
! pip install --upgrade pandas

^C


In [54]:
ratings_matrix=user_ratings_nonan.iloc[:,1:].values


In [55]:
ratings_matrix.shape

(670, 857)

In [56]:
movies_with_overview.head()

,budget,id,popularity,runtime,original_language,title,g_Action,g_Adventure,g_Animation,g_Comedy,...,works,world,worse,writer,wrong,year,years,york,young,younger
0,300000000,285,139.082615,169.0,en,Pirates of the Caribbean: At World's End,1,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,258000000,559,115.699814,139.0,en,Spider-Man 3,1,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,250000000,767,98.885637,153.0,en,Harry Potter and the Half-Blood Prince,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.207012,0.0,0.0,0.000000,0.0
3,200000000,58,145.847379,151.0,en,Pirates of the Caribbean: Dead Man's Chest,1,1,0,0,...,0.388641,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,225000000,2454,53.978602,150.0,en,The Chronicles of Narnia: Prince Caspian,0,1,0,0,...,0.000000,0.0,0.0,0.0,0.0,0.184659,0.0,0.0,0.148494,0.0


In [57]:
movies_encoded=pd.get_dummies(data=movies_with_overview, columns=['original_language'])

In [58]:
movies_copy50=movies_encoded.copy()

In [59]:
movies_encoded

,budget,id,popularity,runtime,title,g_Action,g_Adventure,g_Animation,g_Comedy,g_Crime,...,original_language_it,original_language_ja,original_language_ko,original_language_nl,original_language_pt,original_language_ro,original_language_ru,original_language_sv,original_language_th,original_language_zh
0,300000000,285,139.082615,169.0,Pirates of the Caribbean: At World's End,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,258000000,559,115.699814,139.0,Spider-Man 3,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,250000000,767,98.885637,153.0,Harry Potter and the Half-Blood Prince,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200000000,58,145.847379,151.0,Pirates of the Caribbean: Dead Man's Chest,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,225000000,2454,53.978602,150.0,The Chronicles of Narnia: Prince Caspian,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,380000000,1865,135.413856,136.0,Pirates of the Caribbean: On Stranger Tides,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,180000000,2268,42.990906,113.0,The Golden Compass,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,207000000,254,61.226010,187.0,King Kong,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,200000000,597,100.025899,194.0,Titanic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,200000000,558,35.149586,127.0,Spider-Man 2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
scaler = MinMaxScaler()
movies_matrix=movies_encoded.drop(['title','id'],axis=1)
movies_matrix_scaled=scaler.fit_transform(movies_matrix)

In [75]:
movies_matrix_scaled.shape

(857, 1055)

In [60]:
# movies_matrix=movies_encoded.drop(['title','id'],axis=1)

In [73]:
# movies_matrix_encoded=movies_matrix_scaled.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [78]:
movies_matrix_scaled.shape

(857, 1055)

In [79]:
user_preferences=np.dot(ratings_matrix,movies_matrix_scaled)

In [80]:
row_sums = user_preferences.sum(axis=1)
user_preferences = user_preferences / row_sums[:, np.newaxis]

In [81]:
weighted_features_matrix=np.dot(user_preferences,movies_matrix_encoded.T)

In [82]:
weighted_features_matrix.shape

(670, 857)

In [83]:
ratings_matrix.shape

(670, 857)

In [85]:
req_matrix=np.where(ratings_matrix==0,weighted_features_matrix,-np.inf)

In [94]:
def give_overview_recommendations(user_id,req_matrix):
#     movie_id=movies[user_.title==movie_title]['id'].tolist()[0]
    index=userid_index_mapping[userid_index_mapping.user_id==user_id]['index'].tolist()[0]
#     matrix_req=weighted_features_matrix*tarings_matrix
    movies_rec=sorted(list(enumerate(req_matrix[index])),key=lambda x:x[1],reverse=True)
    recommended_movie_indices=[i[0] for i in movies_rec]
    recommended_movies=[]
    for i in recommended_movie_indices[0:11]:
        movie_id_rec=movieid_index_mapping[movieid_index_mapping.index==i]['movieId'].tolist()[0]
        movie_title_rec=movies[movies.id==movie_id_rec]['title'].tolist()[0]
        recommended_movies.append(movie_title_rec)
    return recommended_movies

In [92]:
userid_index_mapping

,index,user_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
5,5,6
6,6,7
7,7,8
8,8,9
9,9,10


In [95]:
give_overview_recommendations(5,req_matrix)

['Dancer in the Dark',
 'Four Rooms',
 'Star Wars',
 'Finding Nemo',
 'American Beauty',
 'Metropolis',
 'My Life Without Me',
 'Forrest Gump',
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 'Apocalypse Now',
 '8 Mile']

# Content Based Recommendations are made

# Collaborative Filtering

## (1) KNN Neighbors